## Thread Safe Collection
### Legacy
The following thread safe classes existed in Java since 1.0:
- `HashTable`
- `Vector`
- `Stack`

These classes have simple implementation and make use of `synchronized` methods extensively:

In [ ]:
public class Vector<E> {
    public synchronized boolean add(E e) {
        // ...
    }
    
    public synchronized E get(int index) {
        // ...
    }

    public synchronized <T> T[] toArray(T[] a) {
        // ...
    }

    public synchronized int size() {
        // ...
    }

    // other methods
}

Example usage:

In [ ]:
private Vector<Observer> observers;

public void addObserver(Observer observer) {
    observers.add(observer);
}

public void removeObserver(Observer observer) {
    observers.remove(observer);
}

public void notifyAll(Event event) {
    Observer[] observerArray = observers.toArray(new Observer[0]);
    for(Observer observer: observerArray) {
        observer.update(event);
    }
}

There is a race condition in `notifyAll` method. The list of observers can change while iterating over `Observer` array.

### Synchronized Wrappers
The `Collections` class contains multiple methods that return a thread-safe version of non thread-safe classes:
- `Collections.synchronizedSet`
- `Collections.synchronizedSortedSet`
- `Collections.synchronizedNavigableSet`
- `Collections.synchronizedList`
- `Collections.synchronizedMap`
- `Collections.synchronizedSortedMap`
- `Collections.synchronizedNavigableMap`

### Thread Aware Collections
These implementations minimize the need of synchronization. Examples:  
**Copy-On-Write Collections:** on every write (add/remove/set), create a brand new copy of the underlying array. All reads are performed on an immutable, never-changing snapshot. Therefore, readers never block writers, and writers never block readers.
- `CopyOnWriteArrayList`
- `CopyOnWriteArraySet`

In [ ]:
private volatile Object[] array;

public boolean add(E e) {
    synchronized (lock) {
        Object[] oldArray = array;
        oldArray = Arrays.copyOf(oldArray, oldArray.length + 1);
        oldArray[oldArray.length] = e;
        
        array = oldArray;
        return true;
    }
}

public E get(int index) {
    return (E) array[index];
}

Reads are very fast since it doesn't require locks. Writes are slow $O(n)$ since it involves making copy of underlying array. Copy-on-write collections should be preferred when we have small collection with infrequent writes.

**Unbounded Blocking Queues:** `take` and `put` operations block till the queue has elements or has space respectively.
- `LinkedBlockingQueue`
- `LinkedTransferQueue`
- `LinkedBlockingDeque`
- `PriorityBlockingQueue`
- `DelayQueue`
- `SynchronousQueue`

**Bounded Blocking Queues:** similar to previous classes except for capacity constraint.
- `ArrayBlockingQueue`

Example implementation of a linked blocking queue:

In [ ]:
class LinkedBlockingQueue<E> {
    static class Node<E> {
        E item;
        Node<E> next;
        Node(E x) { item = x; }
    }
    private Node<E> head;
    private Node<E> last;
    private final AtomicInteger count = new AtomicInteger();
    private final int capacity;

    private final ReentrantLock putLock = new ReentrantLock();
    private final ReentrantLock takeLock = new ReentrantLock();
    private final Condition notFull = putLock.newCondition();
    private final Condition notEmpty = takeLock.newCondition();

    // Set capacity = Integer.MAX_VALUE for unbounded queue
    public LinkedBlockingQueue(int capacity) {
        this.capacity = capacity;
        head = last = new Node<>(null);
    }

    public boolean put(E e) throws InterruptedException {
        Objects.requireNonNull(e);
        Node<E> node = new Node<>(e);
        int size;
        putLock.lockInterruptibly();
        try {
            // Wait till the queue is not full
            while(count.get() == capacity) {
                notFull.await();
            }

            last = last.next = node; // Add node at the end

            // After adding new node, if there is still capacity left
            // notify other threads that may be waiting in put -- (A)
            size = count.getAndIncrement(); // size is the previous size
            if (size + 1 < capacity) { // size + 1 after new node is added
                notFull.signal();
            }
        } finally {
            putLock.unlock();
        }

        // If previous size was zero, now it would be 1 meaning no longer empty
        // therefore notify other threads that may be waiting in take for 
        // queue to be not empty
        if (size == 0) {
            takeLock.lock();
            try {
                notEmpty.signal();
            } finally {
                takeLock.unlock();
            }
        }
        return true;
    }

    public E take() throws InterruptedException {
        E result;
        int size;
        takeLock.lockInterruptibly();
        try {
            // Wait till there is an element in the queue.
            while (count.get() == 0) {
                notEmpty.await();
            }

            // Remove node from the start.
            Node<E> first = head.next;
            result = first.item;
            first.item = null;
            head = first;

            // Signal other threads waiting on notEmpty condition inside
            // take if there are more than one element in the queue -- (B)
            size = count.getAndDecrement();
            if (size > 1) {
                notEmpty.signal();
            }

        } finally {
            takeLock.unlock();
        }

        // Signal threads waiting inside put that there is empty space in 
        // the queue now
        if (size == capacity) {
            putLock.lock();
            try {
                notFull.signal();
            } finally {
                putLock.unlock();
            }
        }
        return result;
    }
}

Why do we need statements (A) and (B)? Shouldn't a waiting `put` thread be awakened by a `take` thread and vice-versa? A possible [explanation](https://stackoverflow.com/questions/63702223/why-does-linkedblockingqueueput-need-notfull-signal).

`take` and `put` operate on different nodes, therefore both can be guarded by different locks (except for small portion at the end where `take` and `put` compete for the same lock.

`ArrayBlockingQueue` utilises array and therefore has a very different implementation:

In [ ]:
// take and put both access the same array, therefore both the operations are
// guarded by the same lock
class ArrayBlockingQueue<E> {
    private final Object[] items;
    private int count;
    private int putIndex;
    private int takeIndex;

    private final ReentrantLock lock = new ReentrantLock();
    private final Condition notFull = lock.newCondition();
    private final Condition notEmpty = lock.newCondition();

    public ArrayBlockingQueue(int capacity) {
        items = new Object[capacity];
    }

    public void put(E e) throws InterruptedException {
        Objects.requireNonNull(e);
        lock.lockInterruptibly();
        try {
            // Wait till there is space in the queue
            while (count == items.length) {
                notFull.await();
            }

            items[putIndex] = e;
            if (++putIndex == items.length) {
                putIndex = 0;
            }
            count++;
            notEmpty.signal();
        } finally {
            lock.unlock();
        }
    }

    public E take() throws InterruptedException {
        lock.lockInterruptibly();
        try {
            // Wait till there is an element in the queue
            while(count == 0) {
                notEmpty.await();
            }
            
            E result = (E) items[takeIndex];
            if (++takeIndex == items.length) {
                takeIndex = 0;
            }
            count--;
            notFull.signal();
            return result;
        } finally {
            lock.unlock();
        }
    }
}

**Lock-Free Queues:** uses lock-free (CAS loop) implementation described [here](http://www.cs.rochester.edu/~scott/papers/1996_PODC_queues.pdf).
- `ConcurrentLinkedQueue`
- `ConcurrentLinkedDeque`

**Concurrent Maps and Set:** thread safe implementations. `ConcurrentHashMap` for example, ensure non-blocking read (no locks). Writes uses per-bucket locks which means only threads modifying the same bucket contend with each other. Additionally it also provides multiple atomic operations like `putIfAbsent`, `computeIfAbsent`, `merge`, etc.
- `ConcurrentHashMap`
- `ConcurrentSkipListMap`
- `ConcurrentSkipListSet`

Below is a simplistic implementation of `ConcurrentHashMap`:

In [ ]:
// Based on improvements made to ConcurrentHashMap implementation in Java 8
class ConcurrentHashMap<K, V> {
    static class Node<K, V> {
        final K key;
        volatile V value;
        volatile Node<K, V> next;

        Node(K key, V value, Node<K, V> next) {
            this.key = key;
            this.value = value;
            this.next = next;
        }
    }

    private final AtomicReferenceArray<Node<K, V>> table;
    private final int capacity;

    public ConcurrentHashMap(int capacity) {
        this.capacity = capacity;
        table = new AtomicReferenceArray<>(capacity);
    }

    private int index(Object key) {
        return (key.hashCode() & 0x7fffffff) % capacity;
    }

    public V get(K key) {
        int index = index(key);
        Node<K, V> head = table.get(index);

        for( Node<K, V> node = head; node != null; node = node.next) {
            if(node.key.equals(key)) {
                return node.value;
            }
        }

        return null;
    }

    public V put(K key, V value) {
        int index = index(key);
        Node<K, V> head = table.get(index);

        if (head == null) {
            // Use CAS to atomically set the first node
            if (table.compareAndSet(index, null, new Node<>(key, value, null))) {
                return null;
            }
        }
        // CAS failed, another thread added a node, fall through to synchronized block

        head = table.get(index);
        synchronized (head) {
            // Check if head is still the same (hasn't been replaced)
            if (table.get(index) == head) {
                Node<K, V> cur = head;
                while (true) {
                    // Key exists, update value
                    if (cur.key.equals(key)) {
                        V oldValue = cur.value;
                        cur.value = value;
                        return oldValue;
                    }

                    Node<K,V> next = cur.next;
                    // Reached end, add new node
                    if (next == null) {
                        cur.next = new Node<>(key, value, null);
                        return null;
                    }

                    cur = cur.next;
                }
            }
        }

        // Head changed during our operation, retry
        return put(key, value);
    }

    public V remove(K key) {
        int index = index(key);
        Node<K, V> head = table.get(index);
        if (head == null) {
            return null;
        }

        synchronized (head) {
            if (table.get(index) == head) {
                Node<K, V> prev = null;
                Node<K, V> cur = head;
                while (cur != null) {
                    if (cur.key.equals(key)) {
                        // Key exists at head, remove it
                        if (prev == null) {
                            table.set(index, cur.next);
                        // Key exists in the middle or at the end
                        } else {
                            prev.next = cur.next;
                        }
                        return cur.value;
                    }

                    prev = cur;
                    cur = cur.next;
                }

                // Key not found,
                return null;
            }
        }

        // Head changed during our operation, retry
        return remove(key);
    }
}

## Lazy Singleton
Correct implementation:

In [ ]:
class Singleton {
    private static volatile Singleton instance = null;

    public static Singleton getInstance() {
        if (instance == null) {
            synchronized (Singleton.class) {
                if (instance == null)
                    instance = new Singleton();
            }
        }

        return instance;
    }
}

If `instance` was not marked as `volatile` then there is no guarantee that other thread executing `getInstance` would observe the write made.